In [ ]:
import albumentations as A

balancer = A.Compose([
    A.RandomScale(scale_limit=0.1,p=0.5),
    A.RandomCrop(height=400,width=400,p=1.0),
    A.HorizontalFlip(p=0.5),
    A.RGBShift(p=0.5),
    A.RandomBrightness(p=0.5),
    A.RandomContrast(p=0.5)
])

In [ ]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_df = pd.read_csv('../input/identify-the-dance-form/dataset/train.csv')

dance2idx = {dance:i for i,dance in enumerate(set( train_df.values[:,1] ))}
idx2dance = {i:dance for dance,i in dance2idx.items()}

print(dance2idx)
print(idx2dance)

In [ ]:
# plt.subplot(1,2,1)

dances,dance_counts = np.unique(train_df.values[:,1],return_counts=True)
pd.DataFrame({
    'train':dance_counts},
    index=dances
).plot.bar()
plt.show()

plt.subplot(1,2,1)
dances,dance_counts = np.unique(train_df.values[:,1],return_counts=True)
plt.pie(dance_counts,
        labels=dances,
       autopct='%0.1f%%')
plt.title('Proportion of each observed category')
plt.show()

In [ ]:
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import numpy as np
import random

def load_data(base_path,dance2idx):
    data = {file:dance2idx[category] for file,category in train_df.values}
    
    X = []
    y = []
    for filename,category in tqdm(data.items()):
        
        file_path = base_path+filename
        image = load_img(file_path,target_size=(400,400))
        image = img_to_array(image,dtype='uint8')
        
        X.append(image)
        y.append(category)
        
                
    
    for filename,category in tqdm(data.items()):
        
        if category != dance2idx['manipuri']:
            continue
        
        if(y.count(dance2idx['manipuri'])>47):
            break
        
        file_path = base_path+filename
        image = load_img(file_path,target_size=(500,500))
        image = img_to_array(image,dtype='uint8')
        image = balancer(image=image.copy())['image']
        X.append(image)
        y.append(category)
    
    return X,y


    


def define_model(lr=0.001, input_shape=(299,299,3)):
    
    base_model = InceptionV3(include_top=False, weights='imagenet',input_shape=input_shape)

    inp = base_model.layers[-1].output
    inp = layers.Flatten()(inp)
    inp = layers.Dropout(0.5)(inp)
    inp = layers.Dense(1024,activation='relu',kernel_initializer='glorot_normal')(inp)
    inp = layers.Dropout(0.5)(inp)
    inp = layers.Dense(512,activation='relu',kernel_initializer='glorot_normal')(inp)
    output = layers.Dense(8,activation='softmax')(inp)

    model = Model(inputs=base_model.inputs, outputs=output)
    
    opt = Adam(lr=lr)
    model.compile(loss='sparse_categorical_crossentropy' , optimizer=opt,metrics=['acc'])
    
    return model

In [ ]:
import albumentations as A
import random

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

augmenter1 = A.Compose([
    A.RandomScale(scale_limit=0.1,p=0.5),
    A.RandomCrop(height=299,width=299,p=1.0),
    A.HorizontalFlip(p=0.5),
    A.RGBShift(p=0.5),
    A.RandomBrightness(p=0.5),
    A.RandomContrast(p=0.5),
    A.ISONoise(p=0.5),
    A.GaussNoise(p=0.5),
    A.IAAAdditiveGaussianNoise(0.5),
    A.GaussianBlur(p=0.5),
    A.ChannelShuffle(p=0.5),
    A.CoarseDropout(max_holes=5,max_height=40,max_width=40, min_holes=1, min_height=20,fill_value=255, min_width=20, p=0.5)
])

augmenter2 = A.Compose([
    A.RandomScale(scale_limit=0.1,p=0.5),
    A.RandomCrop(height=299,width=299,p=1.0),
    A.HorizontalFlip(p=0.5),
    A.CoarseDropout(max_holes=5,max_height=40,max_width=40, min_holes=1, min_height=20,fill_value=255, min_width=20, p=0.5)
])

testing_aug = A.Compose([
#     A.RandomScale(scale_limit=0.1,p=1.0),
    A.CenterCrop(height=224,width=224,p=1.0),
#     A.RandomCrop(height=224,width=224,p=1.0)
])

def define_generator(x,y,batch_size=32,augmenter=None,pixel=True):
    while True:
        
        combine = list(zip(x,y))
        random.shuffle(combine)
        x,y = zip(*combine)
        
        if pixel==True:
            eraser = get_random_eraser(p=0.9,pixel_level=True)
        else:
            eraser = get_random_eraser(p=0.5,pixel_level=True)
            
        
        n_batchs = len(x) // batch_size
#         print('\n',n_batchs)
        for i in range(n_batchs):
            x_batch = x[i*batch_size:(i+1)*batch_size]
            y_batch = y[i*batch_size:(i+1)*batch_size]
            
            if augmenter is not None:
                x_batch = [augmenter(image=image.copy())["image"] for image in x_batch]
            
            
            x_batch = [preprocess_input(eraser(image)) for image in x_batch]    
            x_batch = np.array(x_batch,dtype=x_batch[0].dtype)
            y_batch = np.array(y_batch)

            yield x_batch,y_batch

In [ ]:
base_path = "../input/identify-the-dance-form/dataset/train/"
trainX,trainY = load_data(base_path,dance2idx)

combine = list(zip(trainX,trainY))
random.shuffle(combine)
trainX,trainY = zip(*combine)

In [ ]:
model = define_model(lr=0.00005)
batch_size = 16
n_batchs = len(trainX)//batch_size

generator = define_generator(trainX,trainY,batch_size=16,augmenter=augmenter1,pixel=False)
model.fit_generator(generator, epochs=100, steps_per_epoch=n_batchs, verbose=1)

In [ ]:
generator = define_generator(trainX,trainY,batch_size=16,augmenter=augmenter1,pixel=False)
model.fit_generator(generator, epochs=100, steps_per_epoch=n_batchs, verbose=1)

In [ ]:
test_df = pd.read_csv("../input/identify-the-dance-form/dataset/test.csv")
base_path = "../input/identify-the-dance-form/dataset/test/"
testX = []

for file in test_df.values[:,0]:
    file_path = base_path+file

    image = load_img(file_path,target_size=(299,299))
    image = img_to_array(image,dtype='uint8')
#     image = testing_aug(image=image)['image']
    image = preprocess_input(image)
    

    testX.append(image)
    
testX = np.array(testX,dtype=testX[0].dtype)
print(testX.shape)

pred = model.predict(testX)

pred = [idx2dance[np.argmax(idx)] for idx in pred]

df = {"Image":test_df.values[:,0] , "target":pred}

print(len(pred))
print(test_df.values[:,0].shape)

df = pd.DataFrame(df)
df.to_csv('submission5.csv',index=False)